In [2]:
import numpy as np
import pandas as pd
import os

# Ruta del archivo
ruta = r"C:\Users\uzgre\Codes\Python\Datathon\Reto Oxxo"
archivo = os.path.join(ruta, "test_w_label.csv")

# Lectura robusta
with open(archivo, 'r', encoding='utf-8', errors='replace') as f:
    Data = pd.read_csv(f)

# Si hay una columna de índice agregada al guardar con index=True, la eliminamos
if 'Unnamed: 0' in Data.columns:
    Data = Data.drop(columns=['Unnamed: 0'])

# Eliminamos columnas innecesarias
Data = Data.drop(columns=['TIENDA_ID', 'DATASET'])

# ===   Tratamos la variable Mts2 por valores faltantes ===

# Primero reemplazamos los ceros por NaN (asumiendo que 0 significa dato faltante)
Data['MTS2VENTAS_NUM'] = Data['MTS2VENTAS_NUM'].replace(0.0, np.nan)

# Calculamos la media global por si hace falta al final
media_global = Data['MTS2VENTAS_NUM'].mean()

# Imputación por grupo: usamos PLAZA_CVE porque puede capturar diferencias regionales
Data['MTS2VENTAS_NUM'] = Data.groupby('PLAZA_CVE')['MTS2VENTAS_NUM'].transform(
    lambda x: x.fillna(x.mean())
)

# Si aún quedan NaNs (por ejemplo, si algún grupo tiene todos los valores nulos), imputamos con la media global
Data['MTS2VENTAS_NUM'] = Data['MTS2VENTAS_NUM'].fillna(media_global)

# ===   Tratamos las variables categóricas ===

# -------- NIVEL SOCIOECONÓMICO (Ordinal Encoding) --------
orden_nivel = {
    'A': 1,
    'AB': 2,
    'B': 3,
    'BC': 4,
    'C': 5,
    'CD': 6,
    'D': 7
}
Data['NIVELSOCIOECONOMICO_DES'] = Data['NIVELSOCIOECONOMICO_DES'].astype(str).map(orden_nivel)

# Checamos si hubo valores no mapeados
n_missing = Data['NIVELSOCIOECONOMICO_DES'].isna().sum()
if n_missing > 0:
    print(f"⚠️ Hay {n_missing} valores de nivel socioeconómico no mapeados. Serán imputados con nivel neutro (4 - BC).")
    Data['NIVELSOCIOECONOMICO_DES'] = Data['NIVELSOCIOECONOMICO_DES'].fillna(4)

# -------- ENTORNO (One-Hot Encoding) --------
Data = pd.get_dummies(Data, columns=['ENTORNO_DES'], prefix='ENTORNO')

# -------- SEGMENTO MAESTRO (One-Hot Encoding + NaN -> 'Missing') --------
Data['SEGMENTO_MAESTRO_DESC'] = Data['SEGMENTO_MAESTRO_DESC'].fillna("Missing")
Data = pd.get_dummies(Data, columns=['SEGMENTO_MAESTRO_DESC'], prefix='SEGMENTO')

# -------- UBICACIÓN TIENDA (One-Hot Encoding) --------
Data = pd.get_dummies(Data, columns=['LID_UBICACION_TIENDA'], prefix='UBICACION')

# -------- Limpieza final --------
Data = Data.drop_duplicates()

# -------- Guardado --------
archivo_salida = os.path.join(ruta, "Dataset_Test_limpio.csv")
Data.to_csv(archivo_salida, index=False)

print("Tamaño del dataset limpio:", Data.shape)
print("✅ Dataset limpio y codificado guardado como 'Dataset_Test_limpio.csv'")

Tamaño del dataset limpio: (100, 22)
✅ Dataset limpio y codificado guardado como 'Dataset_Test_limpio.csv'
